# Setting

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchtext import data, datasets
from torchtext.vocab import Vectors
from torchtext.data import TabularDataset
from torchtext.data import Iterator
from konlpy.tag import Mecab

import sklearn.metrics as metrics
import pandas as pd
import numpy as np
import gensim
import random
import os
import warnings
warnings.filterwarnings(action='ignore')

# 각종 전역변수들
data_path = 'data\\korean-hate-speech\\labeled'
path_train_data = 'data\\korean-hate-speech\\labeled\\train.tsv'
path_dev_data = 'data\\korean-hate-speech\\labeled\\dev.tsv'
BATCH_SIZE = 256
HIDDEN_DIM = 256
NUM_LSTM_LAYER = 2
n_classes = 3
learning_rate = 0.01
MAX_LEN = 80

# 시드 고정
SEED = 5
random.seed(SEED)
torch.manual_seed(SEED)

# CUDA setting 확인
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print("cpu와 cuda 중 다음 기기로 학습함:", DEVICE)

cpu와 cuda 중 다음 기기로 학습함: cuda


# Torchtext dataset 작성

참고자료  
https://wikidocs.net/65348  
https://wikidocs.net/64904  

--------------------------------------

elmo 임베딩
https://github.com/HIT-SCIR/ELMoForManyLangs

## Torchtext를 이용한 전처리

gensim word2vec_kor를 불러내서 torch에서 이용

In [2]:
def preprocess_label(label, only_hate=False):
    if only_hate is True:
        label_matching = {'none':0, 'offensive':1, 'hate':1}
    else:
        label_matching = {'none':0, 'offensive':1, 'hate':2}
    
    return label_matching[label]

def preprocess_bias(bias):
    label_matching = {'none':0, 'others':1, 'gender':2}
    
    return label_matching[bias]

### 필드 정의

In [3]:
tokenizer = Mecab('C:\mecab\mecab-ko-dic')

# comments, hate 만 사용할 거임
comments = data.Field(sequential=True,
                      use_vocab=True,
                      tokenize=tokenizer.morphs,
                      lower=True,
                      batch_first=True, 
                      fix_length = MAX_LEN)

contain_gender_bias = data.Field(sequential=False,
                                 use_vocab=False,
                                 batch_first=True, 
                                 preprocessing=lambda x: x =='True')

bias = data.Field(sequential=False,
                  use_vocab=False,
                  batch_first=True, 
                  preprocessing=lambda x:preprocess_bias(x))

hate = data.Field(sequential=False,
                  use_vocab=False,
                  is_target=True,
                  batch_first=True, 
                  preprocessing=lambda x:preprocess_label(x))

### 본격적인 데이터셋 구성

In [4]:
train_data, val_data = TabularDataset.splits(
    path=data_path, train='train.tsv', validation='dev.tsv', format='tsv',
    fields=[('comments', comments), ('contain_gender_bias', contain_gender_bias), ('bias', bias), ('hate', hate)],
    skip_header=True)

train_data, test_data = train_data.split(split_ratio=0.8)

print('훈련 샘플의 개수 : {}'.format(len(train_data)))
print('검증 샘플의 개수 : {}'.format(len(val_data)))
print('테스트 샘플의 개수 : {}'.format(len(test_data)))
print(vars(train_data[0]))

훈련 샘플의 개수 : 6317
검증 샘플의 개수 : 471
테스트 샘플의 개수 : 1579
{'comments': ['오창', '이채', '은', '이제', '그만', '좀', '나와라', '너무', '설정', '에', '집착', '진심', '도', '없', '구', '~', '너무', '애', '여우', '같', '아', '싫증'], 'contain_gender_bias': False, 'bias': 1, 'hate': 1}


### 단어 집합 만들기

In [5]:
path_word2vec_kor = './data/word2vec_kor/ko.bin'
model_kor_word2vec = gensim.models.Word2Vec.load(path_word2vec_kor)
gensim_to_torch_kor_word2vec = 'torch_kor_word2vec.wv'
model_kor_word2vec.wv.save_word2vec_format(gensim_to_torch_kor_word2vec)

In [6]:
vectors = Vectors(name=gensim_to_torch_kor_word2vec)

In [7]:
comments.build_vocab(train_data, vectors=vectors, min_freq=3, max_size=10000)
#hate.build_vocab(train_data)

print('단어 집합의 크기 : {}'.format(len(comments.vocab)))
print('임베딩 벡터 크기: {}'.format(comments.vocab.vectors.shape))

단어 집합의 크기 : 4033
임베딩 벡터 크기: torch.Size([4033, 200])


### 데이터 로더 만들기

본래 미니 배치 간 샘플 길이가 모두 다르지만, 앞에 torchtext 변수를 선언할때 fix_length를 이용해 통일시켜주었다. CNN에 집어넣어야하기 때문

In [8]:
"""
# 다른 방법
train_loader = Iterator(dataset=train_data, batch_size = BATCH_SIZE)
test_loader = Iterator(dataset=test_data, batch_size = BATCH_SIZE)

print('훈련 데이터의 미니 배치 수 : {}'.format(len(train_loader)))
print('테스트 데이터의 미니 배치 수 : {}'.format(len(test_loader)))
"""

train_iter, val_iter, test_iter = data.BucketIterator.splits(
        (train_data, val_data, test_data), batch_size=BATCH_SIZE,
        shuffle=True, repeat=False, sort_within_batch=False, sort_key=lambda x: len(x.comments))

print('훈련 데이터의 미니 배치의 개수 : {}'.format(len(train_iter)))
print('테스트 데이터의 미니 배치의 개수 : {}'.format(len(test_iter)))
print('검증 데이터의 미니 배치의 개수 : {}'.format(len(val_iter)))

훈련 데이터의 미니 배치의 개수 : 25
테스트 데이터의 미니 배치의 개수 : 7
검증 데이터의 미니 배치의 개수 : 2


# CNN + BiLSTM

In [ ]:
class CNNBiLSTM(nn.Module):
    def __init__(self, pre_embedding, hidden_dim, model_embedding, num_lstm_layer, n_classes, dropout=0.3):
        super(CNNBiLSTM, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.num_lstm_layer = num_lstm_layer
        self.n_classes = n_classes
        # embedding dimension 200
        self.embedding = nn.Embedding.from_pretrained(pre_embedding, freeze=False)
        
        # cnn layer 세팅
        # kernel_sizes = [3, 3]   # kernel size는 홀수로 씀
        # self.convs1 = [nn.Conv2d(1, embedding_dim, (k, embedding_dim), stride=1, padding=(k//2, 0)) for k in kernel_sizes]
        self.cnn_layer1 = nn.Sequential(
            nn.Conv2d(1, self.embedding.embedding_dim, kernel_size=(3, self.embedding.embedding_dim), stride=1, padding=1),
            nn.ReLU())
        self.cnn_layer2 = nn.Sequential(
            nn.Conv2d(self.embedding.embedding_dim, 1, kernel_size=3, stride=1, padding=1),
            nn.ReLU())
        
        # BiLSTM layer 세팅
        self.bi_lstm = nn.LSTM(input_size=3,
                               hidden_size=self.hidden_dim,
                               num_layers=self.num_lstm_layer,
                               dropout=dropout,
                               batch_first=True,
                               bidirectional=True)
        
        #self.maxpool1d = nn.MaxPool1d(kernel_size=2)
        
        # last linear layer 세팅
        # bidirectional 이라서 hidden_dim * 2
        self.linear = nn.Linear(self.hidden_dim * 2, self.n_classes)
                
        
    def forward(self, sents):
        # embedding
        print('sents: ', sents.shape)
        embedded = self.embedding(sents)
        print('emebdded: ', embedded.shape)
        
        # conv2d에 넣기 위해 channel 차원을 늘려준다.
        cnn_x = embedded.unsqueeze(1)
        print('cnn_x: ', cnn_x.shape)
        #print('unsqueeze cnn_x shape: {}'.format(cnn_x.shape))
        # conv2d 통과
        cnn_x = self.cnn_layer1(cnn_x)
        print('cnn_x -> conv2d: ', cnn_x.shape)
        #print('after cnn_layer 1 cnn_x shape: {}'.format(cnn_x.shape))
        cnn_x = self.cnn_layer2(cnn_x)
        print('cnn_x -> conv2d: ', cnn_x.shape)
        #print('after cnn_layer 2 cnn_x shape: {}'.format(cnn_x.shape))
        # lstm에 넣기 위해 channel 차원을 없애준다.
        cnn_x = cnn_x.squeeze(1)
        print('before lstm cnn_x: ', cnn_x.shape)
        #print('after squeeze cnn_x shape: {}'.format(cnn_x.shape))
        break

        # lstm 통과
        lstm_out, (h_n, c_n) = self.bi_lstm(cnn_x)
        
        # forward와 backward의 마지막 time-step의 은닉 상태를 가지고 와서 concat
        # 이때 모델이 batch_first라는 점에 주의한다. (dimension 순서가 바뀜)
        hidden = torch.cat((h_n[-2,:,:], h_n[-1,:,:]), dim = 1)
        out=self.linear(hidden)
        
        return out

# Training function 정의

In [ ]:
def train(model, optimizer, train_iter):
    model.train()
    
    corrects, total_loss = 0, 0
    
    for b, batch in enumerate(train_iter):
        # comments 는 x, hate(label)은 y로 두고
        x, y = batch.comments.to(DEVICE), batch.hate.to(DEVICE)
        # gradient 0으로 세팅해두고
        optimizer.zero_grad()
        # model 돌리고
        prediction = model(x)
        # loss 구해서 backprop
        loss = criterion(prediction, y)
        total_loss = total_loss + loss.item()
        loss.backward()
        optimizer.step()
    
    size = len(train_iter.dataset)
    avg_loss = total_loss / size
    
    return avg_loss
        
def evaluate(model, val_iter):
    model.eval()
    
    corrects, total_loss = 0, 0
    
    for b, batch in enumerate(val_iter):
        x, y = batch.comments.to(DEVICE), batch.hate.to(DEVICE)
        prediction = model(x)
        loss = criterion(prediction, y)
        total_loss = total_loss + loss.item()
        corrects = corrects + (prediction.max(1)[1].view(y.size()).data == y.data).sum()
        
        y = y.data
        y = y.to("cpu")
        y = y.detach().numpy()
        p = prediction.max(1)[1].data
        p = p.to("cpu")
        p = p.detach().numpy()
    
    print('**** Look only last batch case ****')
    print(metrics.classification_report(y,p))
        
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100.0 * corrects / size
    
    return avg_loss, avg_accuracy

# CNN+BiLSTM 선언

In [ ]:
cnn_bilstm_model = CNNBiLSTM(comments.vocab.vectors,
                             HIDDEN_DIM, model_kor_word2vec,
                             NUM_LSTM_LAYER, n_classes)
print(cnn_bilstm_model)
cnn_bilstm_model = cnn_bilstm_model.to(DEVICE)

optimizer = torch.optim.Adam(cnn_bilstm_model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# CNN+BiLSTM training / test

In [ ]:
best_val_loss = None
num_epoch = 10

print('[STAGE] Train')
for i in range(1, num_epoch+1):
    train_loss = train(cnn_bilstm_model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(cnn_bilstm_model, val_iter)
    result = (
        f'[Epoch: {i}/{num_epoch}] train loss : {train_loss:.4f} '
        f'| val loss : {val_loss:.4f} | val accuracy : {val_accuracy:.4f}%'
    )
    print(result)
    break
    
    # 검증 오차가 가장 적은 최적의 모델을 저장
    if not best_val_loss or val_loss < best_val_loss:
        if not os.path.isdir("snapshot"):
            os.makedirs("snapshot")
        torch.save(cnn_bilstm_model.state_dict(), './snapshot/hate_classification_cnn+bilstm.pt')
        best_val_loss = val_loss
        
cnn_bilstm_model.load_state_dict(torch.load('./snapshot/hate_classification_cnn+bilstm.pt'))
test_loss, test_acc = evaluate(cnn_bilstm_model, test_iter)
result = f'테스트 오차: {test_loss:.4f} | 테스트 정확도: {test_acc:.4f}%'
print(result)